In [1]:
# Import all the necessary libraries

import re
import sys
import itertools

import numpy as np
import pandas as pd

from random import choice
from bs4 import BeautifulSoup
from datetime import datetime as dt
from urllib.request import Request, urlopen


%matplotlib inline


In [2]:
# url = 'https://www.wettportal.com/quotenvergleich/surebets/'

url = 'https://www.wettportal.com/lib/ajax/getDatagridContent.php?lang=de&partner=wettportal&contenttype=surebets&offset=00&pagesize=82&period=all'

req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
url = urlopen(req)
content = url.read()

soup = BeautifulSoup(content, 'lxml')


In [3]:
soup

<html><body><article class="container intro">
<div class="frame">
<div class="gutter"><p>Mit unserem Surebet Tool filtern wir minutenaktuell Surebets durch Vergleich der Wettquoten aller Wettanbieter für euch heraus. Bei Surebets wird auf alle möglichen Ausgänge gesetzt und trotzdem ein Profit erzielt, da die Buchmacher in ihren Wahrscheinlichkeitsberechnungen zum Teil weit auseinanderliegen. Mit einem Klick auf unseren Surebet Rechner übernehmt ihr die angegebenen Quoten, tragt nur noch euren Wetteinsatz ein und der Rechner zeigt euch, wie das Geld verteilt werden muss um eure Surebet gewinnen.</p> <p>Das Wetten auf <strong>Surebets</strong> oder auch Arbitrage Wetten funktioniert nur durch Einsätze bei mehreren Anbietern, die Wettquoten werden relativ schnell angeglichen und es empfiehlt sich, schon vorher Wettkonten bei den Topbuchmachern eröffnet zu haben und eine gewisse Geldmenge für euren Einsatz deponiert zu haben.</p> <p>Unser <strong>Surebet Tool</strong> berechnet zur Zeit d

In [4]:
divs = []
sports = []
country = []
leagues = []

def search_the_bet_divs():
    for div in soup.findAll('article', { "class" : "container surebets" }):
        #print(div)
        
        txt = str(div.findAll('h2'))

        re1='.*?'	# Non-greedy match on filler
        re2='(?:[a-z][a-z0-9_]*)'	# Uninteresting: var
        re3='.*?'	# Non-greedy match on filler
        re4='(?:[a-z][a-z0-9_]*)'	# Uninteresting: var
        re5='.*?'	# Non-greedy match on filler
        re6='((?:[a-z][a-z0-9_]*))'	# Variable Name 1

        rg = re.compile(re1+re2+re3+re4+re5+re6,re.IGNORECASE|re.DOTALL)
        m = rg.search(txt)
        if m:
            var1=m.group(1)
            print("("+var1+")"+"\n")
        
        divs.append(div)
    return divs


def search_the_bet_integer(text):
    for div in text.findAll('div', { "class" : 'left-col'}): #"span-class" : "count_all_events"}):
        t = str(div)
        re1='.*?'	# Non-greedy match on filler
        re2='\\d+'	# Uninteresting: int
        re3='.*?'	# Non-greedy match on filler
        re4='\\d+'	# Uninteresting: int
        re5='.*?'	# Non-greedy match on filler
        re6='(\\d+)'	# Integer Number 1

        rg = re.compile(re1+re2+re3+re4+re5+re6,re.IGNORECASE|re.DOTALL)
        m = rg.search(t)
        if m:
            int1=m.group(1)
            return(int(int1))

        
def get_the_profit_values(txt):
        
    re1='.*?'	# Non-greedy match on filler
    re2='([+-]?\\d*\\.\\d+)(?![-+0-9\\.])'	# Float 1

    rg = re.compile(re1+re2,re.IGNORECASE|re.DOTALL)
    m = rg.search(txt)
    if m:
        float1=m.group(1)
        return "("+float1+")"+"\n"

    
    
    
def get_date_of_the_event(txt):
    re1='.*?'	# Non-greedy match on filler
    re2='(20\\.06\\.2018)'	# DDMMYYYY 1

    rg = re.compile(re1+re2,re.IGNORECASE|re.DOTALL)
    m = rg.search(txt)
    if m:
        ddmmyyyy1=m.group(1)
        return ddmmyyyy1

    
    
#divs = search_the_bet_divs()
maxbet = search_the_bet_integer(soup)


print(maxbet)
print(type(maxbet))
print(type(soup))

43
<class 'int'>
<class 'bs4.BeautifulSoup'>


In [5]:

def random_useragent(url):

        HEADERS = {
                        'User-Agent': 'Mozilla/5.0',
                        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
                        'Accept-Encoding': 'gzip, deflate',
                        'Connection': 'close',
                        'DNT': '1'
                }


        UAS = [
                        'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.97 Safari/537.11',
                        'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:17.0) Gecko/20100101 Firefox/17.0',
                        'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_8_2) AppleWebKit/536.26.17 (KHTML, like Gecko) Version/6.0.2 Safari/536.26.17',
                        'Mozilla/5.0 (Linux; U; Android 2.2; fr-fr; Desire_A8181 Build/FRF91) App3leWebKit/53.1 (KHTML, like Gecko) Version/4.0 Mobile Safari/533.1',
                        'Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; SV1; FunWebProducts; .NET CLR 1.1.4322; PeoplePal 6.2)',
                        'Mozilla/5.0 (Windows NT 5.1; rv:13.0) Gecko/20100101 Firefox/13.0.1',
                        'Opera/9.80 (Windows NT 5.1; U; en) Presto/2.10.289 Version/12.01',
                        'Mozilla/5.0 (Windows NT 5.1; rv:5.0.1) Gecko/20100101 Firefox/5.0.1',
                        'Mozilla/4.0 (compatible; MSIE 8.0; Windows NT 6.0; Trident/4.0; Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; SV1) ; .NET CLR 3.5.30729)'
                ]

        req = Request(url)
        HEADERS['User-Agent'] = choice(UAS)
        req.add_header('User-Agent', HEADERS)

        response = urlopen(req)
        html = response.read()

        return html

In [8]:
url = str('https://www.wettportal.com/lib/ajax/getDatagridContent.php?lang=de&partner=wettportal&contenttype=surebets&offset=00&pagesize='+ str(maxbet) + '&period=all')
print(url)

req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
url = urlopen(req)
content = url.read()


# function to remove the whitespaces from a string - here the name of the event
def strip_list_noempty(mylist):
    newlist = (item.strip() if hasattr(item, 'strip') else item for item in mylist)
    return [item for item in newlist if item != '']


#content = random_useragent(url)

soup = BeautifulSoup(content, 'lxml')

result = []


for div in soup.findAll('article', { "class" : "container surebets" }):
    
    #print(div)
    
    # get the time of the event
    eventtime = div.find('time').get_text()
        
    # get the event (teams, sport, etc.)
    l = div.find('span').get_text().split("|")
    l = strip_list_noempty(l)
    
    
    sport = l[0]
    type1 = l[1]
    type2 = l[2]
    
    print("sport: ", sport)
    print("type1: ", type1)
    print("type2: ", type2)
    
    # get the profitfactor and the teams of the event
    
    p = [y.find('span') for x in div.find('tbody') for y in x]
#   output:
#   a list of a lot empty items (None, -1)
#   the important stuff are two entries:
#   team name/ player / athlete:
#   <span itemprop="name">Argentinien - Guatemala</span>
#   
#   profitfactor:
#   <span class="hilite-red">51.36%</span>

    NoneType = type(None)
    
    teams_and_profitfactor = [i for i, j in enumerate(p) if j if not isinstance(j, int) or isinstance(j, NoneType)]
    print("###1 ",teams_and_profitfactor)
    
    # check is the game is very profitable
   
    if len(teams_and_profitfactor) == 2:
        teams = p[teams_and_profitfactor[0]].get_text()
        print("###2 ", teams)
    
        teams = list(teams.split("-"))
        print("###3 ", teams)
    
        team1 = teams[0]
        print("###4 ", team1)
    
        team2 = teams[1]
        print("###5 ", team2)
        print("####test", p)
    
        profitfactor = p[teams_and_profitfactor[1]].get_text()
    
        print(p)
    
        print("teams: ",teams)
        print("team1: ",team1)
        print("team2: ",team2)
        print("profitfactor: ", profitfactor)

        #for i in p:
        #    print(type(i))
        #print(p)   
    
        # put it all together for one line of data
        data = [eventtime, profitfactor, team1, team2, sport, type1, type2]
        result.append(data)
    
        print(data)
    else:
        print("no more really profitable games...")
    
# check it line by line    
for i in result:
    print(i)
    break
    
 


https://www.wettportal.com/lib/ajax/getDatagridContent.php?lang=de&partner=wettportal&contenttype=surebets&offset=00&pagesize=43&period=all
sport:  Boxen
type1:  International
type2:  Schwergewicht
###1  [2]
no more really profitable games...
sport:  Fussball
type1:  International
type2:  WM 2018 - Gruppe A
###1  [2]
no more really profitable games...
sport:  Tischtennis
type1:  International
type2:  Challenger Series, Männer
###1  [2]
no more really profitable games...
sport:  Tennis
type1:  International
type2:  Wimbledon, Qualifikation - Herren
###1  [2]
no more really profitable games...
sport:  Fussball
type1:  International
type2:  WM 2018 - Gruppe B
###1  [2]
no more really profitable games...
sport:  Fussball
type1:  International
type2:  WM 2018 - Gruppe D
###1  [2]
no more really profitable games...
sport:  Volleyball
type1:  International
type2:  Mittelmeerspiele - Frauen
###1  [2]
no more really profitable games...
sport:  Fussball
type1:  International
type2:  WM 2018 - Gr